In [1]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import requests
import os
from bs4 import BeautifulSoup
from typing import List
from openai import OpenAI
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.knowledge import Knowledge
from agno.vectordb.lancedb import LanceDb, SearchType
from agno.db.sqlite import SqliteDb

load_dotenv()

class CloudRuEmbedder:
    def __init__(self):
        self.client = OpenAI(
            base_url=os.getenv("CLOUD_BASE_URL"),
            api_key=os.getenv("OPENAI_API_KEY"),
        )
        self.dimensions = int(os.getenv("EMBED_DIMENSIONS", "1024"))
        self.model = os.getenv("EMBEDED_MODEL", "Qwen/Qwen3-Embedding-0.6B")

    def embed(self, text: str) -> List[float]:
        resp = self.client.embeddings.create(model=self.model, input=[text])
        return resp.data[0].embedding

article_url = "https://pmjournal.ru/articles/biznes-stati/kratkoe-soderzhanie-pmbok-7/"
response = requests.get(article_url)
response.raise_for_status()
soup = BeautifulSoup(response.content, "html.parser")
article_text = "\n".join(p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip())

embedder = CloudRuEmbedder()
knowledge = Knowledge(
    vector_db=LanceDb(
        uri="./lancedb_pmbok_web",
        table_name="pmbok_web_docs",
        embedder=embedder,
        search_type=SearchType.hybrid,
    )
)

if article_text.strip():
    asyncio.run(knowledge.add_contents_async(texts=[article_text]))

db = SqliteDb(db_file="pmbok_web_agent_memory.db")
model = OpenAILike(
    id="Qwen/Qwen3-235B-A22B-Instruct-2507",
    base_url=os.getenv("CLOUD_BASE_URL").rstrip("/") + "/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

agent = Agent(
    model=model,
    db=db,
    name="PMBOK Web Assistant",
    instructions=[
        "Ты — эксперт по PMBOK Guide 7, основанный на статье с pmjournal.ru.",
        "Отвечай строго по информации из этой статьи.",
        "Если вопрос выходит за её рамки — ищи в интернете через DuckDuckGo.",
        "Всегда отвечай на русском языке."
    ],
    knowledge=knowledge,
    tools=[DuckDuckGoTools(enable_news=True)],
    enable_user_memories=True,
    reasoning=True,
    search_knowledge=True,
    markdown=True,
)

user_input = input("Вы: ").strip()
if user_input:
    agent.print_response(user_input, stream=True, markdown=False)

Output()